In [ ]:
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import time
from torchsummary import summary
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
from torchvision.utils import save_image
from tqdm import tqdm


# Image Augmentation

Augment images to be in line with the pre-trained network format

In [ ]:
image_transforms = { 
    'train': transforms.Compose([
        transforms.Resize(size=227),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    
    'test': transforms.Compose([
        transforms.Resize(size=227),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}



# load all images in train and test sets
dataset = 'Images2'

trainDirectory = os.path.join(dataset, 'Train')
testDirectory = os.path.join(dataset, 'Test')
batchSize = 32

num_classes = len(os.listdir(trainDirectory))

data = {
    'train': datasets.ImageFolder(root=trainDirectory, transform=image_transforms['train']),
    'test': datasets.ImageFolder(root=testDirectory, transform=image_transforms['test'])
}


idx_to_class = {v: k for k, v in data['train'].class_to_idx.items()}


train_data_size = len(data['train'])
test_data_size = len(data['test'])

train_data_loader = DataLoader(data['train'], batch_size=batchSize, shuffle=True)
test_data_loader = DataLoader(data['test'], batch_size=batchSize, shuffle=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# Loss Function

In [ ]:
def BCELoss_class_weighted(weights):

    """Calculate the binary cross entropy with weights """

    def loss(input, target):
        input = torch.clamp(input,min=1e-7,max=1-1e-7)
        bce = - weights[1] * target * torch.log(input) - (1 - target) * weights[0] * torch.log(1 - input)
        return torch.mean(bce)

    return loss

# Initialise Model 

In [ ]:
# load the pretrained weights
alexnet = models.alexnet(weights='IMAGENET1K_V1')

# If both are commented out all layers are retrained

"""
for param in alexnet.parameters():
    param.requires_grad = False       """                # Retrain only last


"""
for name, param in alexnet.named_parameters():
    if 'features' in name:                              # Retrain fully connected layers
        param.requires_grad = False"""


# alter network
num_features = alexnet.classifier[6].in_features
alexnet.classifier[6] = nn.Sequential(
    nn.Linear(num_features, 1),
    nn.Sigmoid()
)



loss_func = BCELoss_class_weighted([1,2.8])
optimizer = optim.Adam(alexnet.parameters())


# Train and Evaluate

In [ ]:
from sklearn.metrics import confusion_matrix

def predict(model,confusion=True):

    """Function to predict the test set and print performance metrics"""

    test_acc = 0.0
    all_predictions = []
    all_labels = []
    
        
    with torch.no_grad():

        for i, (inputs, labels) in enumerate(test_data_loader):

            model.eval()
            outputs = model(inputs)
            predictions = (outputs >= 0.50).float() #torch.round(outputs)
            correct_counts = predictions.eq(labels.data.view_as(predictions))
            acc = torch.mean(correct_counts.type(torch.FloatTensor))
            test_acc += acc.item() * inputs.size(0)

            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

          
    avg_test_acc = test_acc/test_data_size
    

    print("Test Accuracy: ", avg_test_acc)
    

    conf_matrix = confusion_matrix(all_labels, all_predictions)
    

    print("Test Recall: ",conf_matrix[1][1] / (conf_matrix[1][1] + conf_matrix[1][0]) )
    print("Test Precision: ",conf_matrix[1][1] / (conf_matrix[1][1] + conf_matrix[0][1]))
    if confusion == True:
        print("Confusion Matrix:")
        print(conf_matrix)




In [ ]:
def train_and_validate(model, loss_criterion, optimizer, epochs=20):  

    """Function to train and evaluate the model"""


  
    history = []

    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, epochs))
        
        # Training mode
        model.train()
        
        # Loss and accuracy
        train_loss = 0.0
        train_acc = 0.0
    
        
        for i, (inputs, labels) in enumerate(tqdm(train_data_loader)):

            inputs = inputs.to(device)
            labels = labels.to(device)

            labels = labels.unsqueeze(1)
            labels = labels.float()
            
            # clean existing gradients
            optimizer.zero_grad()
            
            # Forward pass to calculate outputs
            outputs = model(inputs)
            
            # Compute loss
            loss = loss_criterion(outputs, labels)
            
            # Backpropagation
            loss.backward()
            
            # Update the weights
            optimizer.step()
            
            # Compute loss for the batch 
            train_loss += loss.item() * inputs.size(0)

            predictions =   (outputs >= 0.50).float() #torch.round(outputs)
        
            correct_counts = predictions.eq(labels.data.view_as(predictions))
            
            # calculate training accuracy
            acc = torch.mean(correct_counts.type(torch.FloatTensor))
            train_acc += acc.item() * inputs.size(0)
            

        # Find average training loss and training accuracy
        avg_train_loss = train_loss/train_data_size 
        avg_train_acc = train_acc/train_data_size


        history.append([avg_train_loss, avg_train_acc])
                
        epoch_end = time.time()
    
        print("Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(epoch+1, avg_train_loss, avg_train_acc*100, epoch_end-epoch_start))
        predict(model,confusion=True)
     
    return model, history


In [ ]:
num_epochs = 20
trained_model, history = train_and_validate(alexnet, loss_func, optimizer, num_epochs)